In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM , LlamaForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-chat-hf")
model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-13b-chat-hf")





Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:08<00:00,  2.93s/it]


In [4]:
prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

"Hey, are you conscious? Can you talk to me?\n\nSure thing! I'm a large language model, so I can"

In [9]:
import os
import re
print(os.getcwd())

/common/home/hg343/Research/LLM_ProblemConversion/src


In [ ]:
problem_path = "/common/home/hg343/Research/LLM_ProblemConversion/datasets/problems/Math_manual"
ground_truth_path = "/common/home/hg343/Research/LLM_ProblemConversion/datasets/solution/ground_truth_Math_manual"
solution_path_noise = "/common/home/hg343/Research/LLM_ProblemConversion/datasets/solution/Wikipedia"
def strip_and_normalize(content):
    return re.sub(r'\s+', ' ', content).strip().lower()

unique_contents = set()
for txt_file in os.listdir(ground_truth_path):
    if txt_file.endswith(".txt"):
        file_path = os.path.join(ground_truth_path, txt_file)
        with open(file_path, 'r') as file:
            file_content = strip_and_normalize(file.read())
            unique_contents.add(file_content)
hint_pool = ", ".join(unique_contents)
# print(hint_pool)
print(hint_pool)

template = f"""You are an expert at mathematics. Your job is to select the three best theorems most applicable for solving the given problem.
 You do not need to solve the problem. You must select your answer from this list of theorems: {hint_pool}{{question}}"""

# Process problems
solution_path = "./../datasets/inference_result"
hit_count, total_files = 0, 0
for problem in os.listdir(problem_path):
    if problem.endswith(".txt"):
        total_files += 1
        file_path = os.path.join(problem_path, problem)
        try:
            with open(file_path, 'r') as file:
                problem_txt = file.read()
        except IOError:
            print(f"Error reading '{file_path}'.")
            continue
        print('ready to prompt')
        question = f"What 3 theorems are best applicable for solving following question? Please provide response that are separated by , only. \n{{\n{problem_txt}\n}}\n"
        prompt  =template + question
        inputs = tokenizer(prompt, return_tensors="pt")
        print('input prepared')
        # Generate
        generate_ids = model.generate(inputs.input_ids, max_length=1000)
        print('generation done')
        response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
        print(response)


menelaus's theorem, niven's theorem, ﻿pythagorean theorem, rédei's theorem, pitot theorem, prime number theorem, binomial theorem, exterior angle theorem, rational root theorem, miquel's theorem, dirichlet's approximation theorem, pythagorean theorem, descartes's theorem, pick's theorem, ﻿de moivre’s theorem, bayes' theorem, ﻿dirichlet’s theorem on arithmetic progressions, ﻿ceva’s theorem, de gua's theorem, ﻿vieta’s formulas, ﻿angle bisector theorem, ﻿binomial theorem, ﻿descartes’ theorem, viviani's theorem, ﻿bezout’s theorem, squeeze theorem, brianchon's theorem, fermat's little theorem, thales's theorem, intermediate value theorem, wilson's theorem, ﻿chinese remainder theorem, morley's trisector theorem
ready to prompt
input prepared
generation done
You are an expert at mathematics. Your job is to select the three best theorems most applicable for solving the given problem.
 You do not need to solve the problem. You must select your answer from this list of theorems: menelaus's theor

In [ ]:
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

sequences = pipeline(
    'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


In [3]:
def get_some_feedbacks(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    # Generate
    generate_ids = model.generate(inputs.input_ids, max_length=30)
    return tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    

In [5]:
print(get_some_feedbacks('wat u doin?'))

Hey, are you conscious? Can you talk to me?

Sure, I can talk to you! I'm a large language


In [ ]:
#The goal is to run the tests 
def hint_generation_llama2(problem_path,ground_truth_path,solution_space_path, model):
    #########
    model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-13b-chat-hf")
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-chat-hf")

############
    # Collect subdirectories
    subdirectories = [item for item in os.listdir(solution_space_path)
                      if os.path.isdir(os.path.join(solution_space_path, item))]
    #  print(", ".join(subdirectories))

    # Function to strip and normalize content
    def strip_and_normalize(content):
        return re.sub(r'\s+', ' ', content).strip().lower()

    # Process ground truths
    unique_contents = set()
    for txt_file in os.listdir(ground_truth_path):
        if txt_file.endswith(".txt"):
            file_path = os.path.join(ground_truth_path, txt_file)
            with open(file_path, 'r') as file:
                file_content = strip_and_normalize(file.read())
                unique_contents.add(file_content)
    hint_pool = ", ".join(unique_contents)
    # print(hint_pool)


    template = f"""You are an expert at mathematics. Your job is to select the three best theorems most applicable for solving the given problem.
     You do not need to solve the problem. You must select your answer from this list of theorems: {hint_pool}{{question}}"""

    # Process problems
    solution_path = "./../datasets/inference_result"
    hit_count, total_files = 0, 0
    for problem in os.listdir(problem_path):
        if problem.endswith(".txt"):
            total_files += 1
            file_path = os.path.join(problem_path, problem)
            try:
                with open(file_path, 'r') as file:
                    problem_txt = file.read()
            except IOError:
                print(f"Error reading '{file_path}'.")
                continue
            question = f"What 3 theorems are best applicable for solving following question? Please provide response that are separated by , only. \n{{\n{problem_txt}\n}}\n"
            prompt  =template + question
            inputs = tokenizer(prompt, return_tensors="pt")
            # Generate
            generate_ids = model.generate(inputs.input_ids, max_length=1000)
            response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
            # Save response to file
            output_file_path = os.path.join(solution_path, problem)
            os.makedirs(solution_path, exist_ok=True)
            try:
                with open(output_file_path, 'w') as file:
                    file.write(response)
                    print("Wrote to: " + output_file_path)
            except IOError:
                print(f"Error writing to '{output_file_path}'.")

            # Compare with ground truth
            ground_truth_file_path = os.path.join(ground_truth_path, problem)
            if os.path.isfile(ground_truth_file_path):
                with open(ground_truth_file_path, 'r',encoding='utf-8-sig') as ground_truth_file:
                    ground_truth_items = [strip_and_normalize(item) for item in ground_truth_file.read().split(',')]
                    response =[strip_and_normalize(item) for item in response.split(',')]
                for item in ground_truth_items:
                    item = item.strip('\'')
                    print('ground_truth_items: ', ground_truth_items)
                    print('response: ', response)
                    if item in response:
                        hit_count += 1
                        logging.info('response contains ground truth', 'hit_count:', hit_count)
                        break
                    else:
                        logging.info('response does not include ground_truth')

    logging.info('hit count: ',hit_count,'total problems: ',total_files)
    return f"{hit_count}/{total_files}"



In [ ]:
problem_path = "/common/home/hg343/Research/LLM_ProblemConversion/datasets/problems/Math_manual"
ground_truth_path = "/common/home/hg343/Research/LLM_ProblemConversion/datasets/solution/ground_truth_Math_manual"
solution_path_noise = "/common/home/hg343/Research/LLM_ProblemConversion/datasets/solution/Wikipedia"
hint_accuracy = hint_generation_llama2(problem_path,ground_truth_path,solution_path_noise, model=None)